In [1]:
import warnings
warnings.filterwarnings("ignore")
from numpy import asarray
import numpy as np
import pandas as pd
from PIL import Image
import cv2
import glob
import os 
import random
import subprocess
import matplotlib.pyplot as plt
from skimage.io import imread
from matplotlib.patches import Rectangle
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, Input, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy

In [2]:
def is_image_readable(file_path):
    try:
        with Image.open(file_path) as img:
            img.verify()
        return True
    except (IOError, SyntaxError) as e:
        return False

In [3]:
image_list_other = []
image_list_safe = []
image_list_talking = []
image_list_text = []
image_list_turn = []

for other in os.listdir("Revitsone-5classes/Revitsone-5classes/other_activities"): 
    if other.endswith(".png") or other.endswith(".jpg"): 
        image_list_other.append(os.path.join("Revitsone-5classes/Revitsone-5classes/other_activities",
                                             other))
        print(os.path.join("Revitsone-5classes/Revitsone-5classes/other_activities", other))
        
for safe in os.listdir("Revitsone-5classes/Revitsone-5classes/safe_driving"): 
    if safe.endswith(".png") or safe.endswith(".jpg"): 
        image_list_safe.append(os.path.join("Revitsone-5classes/Revitsone-5classes/safe_driving", 
                                            safe))
        print(os.path.join("Revitsone-5classes/Revitsone-5classes/safe_driving",
                           safe))
        
for talking in os.listdir("Revitsone-5classes/Revitsone-5classes/talking_phone"): 
    if talking.endswith(".png") or talking.endswith(".jpg"): 
        image_list_talking.append(os.path.join("Revitsone-5classes/Revitsone-5classes/talking_phone", 
                                               talking))
        print(os.path.join("Revitsone-5classes/Revitsone-5classes/talking_phone",
                           talking))
        
for text in os.listdir("Revitsone-5classes/Revitsone-5classes/texting_phone"): 
    if text.endswith(".png") or text.endswith(".jpg"): 
        image_list_text.append(os.path.join("Revitsone-5classes/Revitsone-5classes/texting_phone",
                                            text))
        print(os.path.join("Revitsone-5classes/Revitsone-5classes/texting_phone",
                           text))
        
for turn in os.listdir("Revitsone-5classes/Revitsone-5classes/turning"): 
    if turn.endswith(".png") or turn.endswith(".jpg"): 
        image_list_turn.append(os.path.join("Revitsone-5classes/Revitsone-5classes/turning",
                                            turn))
        print(os.path.join("Revitsone-5classes/Revitsone-5classes/turning", 
                           turn))
    
    

Revitsone-5classes/Revitsone-5classes/other_activities\2019-04-2416-05-13.png
Revitsone-5classes/Revitsone-5classes/other_activities\2019-04-2416-06-20.png
Revitsone-5classes/Revitsone-5classes/other_activities\2019-04-2416-06-37.png
Revitsone-5classes/Revitsone-5classes/other_activities\2019-04-2416-06-59.png
Revitsone-5classes/Revitsone-5classes/other_activities\2019-04-2416-07-58.png
Revitsone-5classes/Revitsone-5classes/other_activities\2019-04-2416-08-09.png
Revitsone-5classes/Revitsone-5classes/other_activities\2019-04-2416-08-22.png
Revitsone-5classes/Revitsone-5classes/other_activities\2019-04-2416-08-44.png
Revitsone-5classes/Revitsone-5classes/other_activities\2019-04-2416-33-00.png
Revitsone-5classes/Revitsone-5classes/other_activities\2019-04-2416-33-09.png
Revitsone-5classes/Revitsone-5classes/other_activities\2019-04-2416-33-19.png
Revitsone-5classes/Revitsone-5classes/other_activities\2019-04-2416-33-23.png
Revitsone-5classes/Revitsone-5classes/other_activities\2019-04-2

In [4]:
for i in image_list_other:
    if is_image_readable(i):
        continue
    else:
        print(f"The image file '{i}' is not readable.")
        image_list_other.remove(i)
for i in image_list_safe:
    if is_image_readable(i):
        continue
    else:
        print(f"The image file '{i}' is not readable.")
        image_list_safe.remove(i)
for i in image_list_talking:
    if is_image_readable(i):
        continue
    else:
        print(f"The image file '{i}' is not readable.")
        image_list_talking.remove(i)
for i in image_list_text:
    if is_image_readable(i):
        continue
    else:
        print(f"The image file '{i}' is not readable.")
        image_list_text.remove(i)
for i in image_list_turn:
    if is_image_readable(i):
        continue
    else:
        print(f"The image file '{i}' is not readable.")
        image_list_turn.remove(i)

The image file 'Revitsone-5classes/Revitsone-5classes/other_activities\img_13318.jpg' is not readable.
The image file 'Revitsone-5classes/Revitsone-5classes/other_activities\img_13396.jpg' is not readable.
The image file 'Revitsone-5classes/Revitsone-5classes/other_activities\img_13541.jpg' is not readable.
The image file 'Revitsone-5classes/Revitsone-5classes/other_activities\img_13625.jpg' is not readable.
The image file 'Revitsone-5classes/Revitsone-5classes/other_activities\img_20398.jpg' is not readable.
The image file 'Revitsone-5classes/Revitsone-5classes/other_activities\img_22266.jpg' is not readable.
The image file 'Revitsone-5classes/Revitsone-5classes/other_activities\img_4664.jpg' is not readable.
The image file 'Revitsone-5classes/Revitsone-5classes/other_activities\img_79.jpg' is not readable.
The image file 'Revitsone-5classes/Revitsone-5classes/other_activities\img_7973.jpg' is not readable.
The image file 'Revitsone-5classes/Revitsone-5classes/turning\img_101434.jpg' 

In [5]:
font = {'family':'Times New Roman','color':'#1f211f'}
background_color = '#fab72f'

In [6]:
print("Train","Test", "Valid")

train_other = image_list_other[:1589]
test_other = image_list_other[1589:2012]
valid_other = image_list_other[2012:]

print (len(train_other), len(test_other), len(valid_other))

train_safe = image_list_safe[:1652]
test_safe = image_list_safe[1652:2092]
valid_safe = image_list_safe[2092:]

print (len(train_safe), len(test_safe), len(valid_safe))

train_talking = image_list_talking[:1626]
test_talking = image_list_talking[1626:2059]
valid_talking = image_list_talking[2059:]

print (len(train_talking), len(test_talking), len(valid_talking))

train_text = image_list_text[:1652]
test_text = image_list_text[1652:2092]
valid_text = image_list_text[2092:]

print (len(train_text), len(test_text), len(valid_text))

train_turn = image_list_turn[:1547]
test_turn = image_list_turn[1547:1959]
valid_turn = image_list_turn[1959:]

print (len(train_turn), len(test_turn), len(valid_turn))

Train Test Valid
1589 423 107
1652 440 111
1626 433 110
1652 440 111
1547 412 98


In [7]:
train_other_df = pd.DataFrame({'image':train_other, 'label':'Other'})
train_safe_df = pd.DataFrame({'image':train_safe, 'label':'Safe'})
train_talking_df = pd.DataFrame({'image':train_talking, 'label':'Talk'})
train_text_df = pd.DataFrame({'image':train_text, 'label':'Text'})
train_turn_df = pd.DataFrame({'image':train_turn, 'label':'Turn'})
test_other_df = pd.DataFrame({'image':test_other, 'label':'Other'})
test_safe_df = pd.DataFrame({'image':test_safe, 'label':'Safe'})
test_talking_df = pd.DataFrame({'image':test_talking, 'label':'Talk'})
test_text_df = pd.DataFrame({'image':test_text, 'label':'Text'})
test_turn_df = pd.DataFrame({'image':test_turn, 'label':'Turn'})
valid_other_df = pd.DataFrame({'image':valid_other, 'label':'Other'})
valid_safe_df = pd.DataFrame({'image':valid_safe, 'label':'Safe'})
valid_talking_df = pd.DataFrame({'image':valid_talking, 'label':'Talk'})
valid_text_df = pd.DataFrame({'image':valid_text, 'label':'Text'})
valid_turn_df = pd.DataFrame({'image':valid_turn, 'label':'Turn'})

In [8]:
train_df = pd.concat([train_other_df, train_safe_df, train_talking_df, train_text_df, train_turn_df])
test_df = pd.concat([test_other_df, test_safe_df, test_talking_df, test_text_df, test_turn_df])
val_df = pd.concat([valid_other_df, valid_safe_df, valid_talking_df, valid_text_df, valid_turn_df])

In [9]:
train_df.head()

,image,label
0,Revitsone-5classes/Revitsone-5classes/other_ac...,Other
1,Revitsone-5classes/Revitsone-5classes/other_ac...,Other
2,Revitsone-5classes/Revitsone-5classes/other_ac...,Other
3,Revitsone-5classes/Revitsone-5classes/other_ac...,Other
4,Revitsone-5classes/Revitsone-5classes/other_ac...,Other


In [10]:
random_img_height = random.choice(train_other)

In [11]:
image= cv2.imread(random_img_height)

height, width= image.shape[:2]

print("The height is ", height)

print("The width is ", width)

The height is  480
The width is  640


In [12]:
Batch_size = 64
Img_height = 240
Img_width = 240

In [13]:
trainGenerator = ImageDataGenerator(rescale=1./255.)
valGenerator = ImageDataGenerator(rescale=1./255.)
testGenerator = ImageDataGenerator(rescale=1./255.)

In [14]:
trainDataset = trainGenerator.flow_from_dataframe(
  dataframe=train_df,
  class_mode="categorical",
  x_col="image",
  y_col="label",
  batch_size=Batch_size,
  seed=42,
  shuffle=True,
  target_size=(Img_height,Img_width) #set the height and width of the images
)

testDataset = testGenerator.flow_from_dataframe(
  dataframe=test_df,
  class_mode='categorical',
  x_col="image",
  y_col="label",
  batch_size=Batch_size,
  seed=42,
  shuffle=True,
  target_size=(Img_height,Img_width)
)

valDataset = valGenerator.flow_from_dataframe(
  dataframe=val_df,
  class_mode='categorical',
  x_col="image",
  y_col="label",
  batch_size=Batch_size,
  seed=42,
  shuffle=True,
  target_size=(Img_height,Img_width)
)

Found 8066 validated image filenames belonging to 5 classes.
Found 2148 validated image filenames belonging to 5 classes.
Found 537 validated image filenames belonging to 5 classes.


In [19]:
model=keras.Sequential([
    keras.layers.Conv2D(100,3,input_shape=(240,240,3),padding='valid',activation='relu'),
    keras.layers.MaxPooling2D(),
    keras.layers.Conv2D(50,3,padding='valid',activation='relu'),
    keras.layers.MaxPooling2D(),
    keras.layers.Conv2D(25,3,padding='valid',activation='relu'),
    keras.layers.MaxPooling2D(),
    keras.layers.Flatten(),
    keras.layers.Dense(32,activation='relu'),
    keras.layers.Dense(5,activation='softmax'),
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.fit(trainDataset,epochs=10)

Epoch 1/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 283s 2s/step - accuracy: 0.4108 - loss: 1.3263
Epoch 2/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 204s 2s/step - accuracy: 0.9092 - loss: 0.3050
Epoch 3/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 205s 2s/step - accuracy: 0.9596 - loss: 0.1336
Epoch 4/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 204s 2s/step - accuracy: 0.9800 - loss: 0.0537
Epoch 5/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 204s 2s/step - accuracy: 0.9835 - loss: 0.0518
Epoch 6/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 203s 2s/step - accuracy: 0.9812 - loss: 0.0559
Epoch 7/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 204s 2s/step - accuracy: 0.9870 - loss: 0.0369
Epoch 8/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 204s 2s/step - accuracy: 0.9950 - loss: 0.0205
Epoch 9/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 218s 2s/step - accuracy: 0.9934 - loss: 0.0181
Epoch 10/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 215s 2s/step - accuracy: 0.9971 - loss: 0.0058


In [20]:
model.evaluate(testDataset)

34/34 ━━━━━━━━━━━━━━━━━━━━ 14s 417ms/step - accuracy: 0.9885 - loss: 0.0412


[0.04285335913300514, 0.9888268113136292]

In [21]:
model.evaluate(valDataset)

9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 376ms/step - accuracy: 0.9922 - loss: 0.0179


[0.03077264130115509, 0.9869646430015564]

In [22]:
def AlexNet(input_shape=(227, 227, 3), num_classes=1000):
    model = keras.Sequential()
    
    # 1st Convolutional Layer
    model.add(keras.layers.Conv2D(filters=96, kernel_size=(11, 11), strides=(4, 4), activation='relu', input_shape=input_shape))
    model.add(keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

    # 2nd Convolutional Layer
    model.add(keras.layers.Conv2D(filters=256, kernel_size=(5, 5), strides=(1, 1), activation='relu', padding='same'))
    model.add(keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

    # 3rd Convolutional Layer
    model.add(keras.layers.Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding='same'))

    # 4th Convolutional Layer
    model.add(keras.layers.Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding='same'))

    # 5th Convolutional Layer
    model.add(keras.layers.Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding='same'))
    model.add(keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

    # Flatten the output of the last pooling layer
    model.add(keras.layers.Flatten())

    # 1st Fully Connected Layer
    model.add(keras.layers.Dense(4096, activation='relu'))
    model.add(keras.layers.Dropout(0.5))

    # 2nd Fully Connected Layer
    model.add(keras.layers.Dense(4096, activation='relu'))
    model.add(keras.layers.Dropout(0.5))

    # Output Layer
    model.add(keras.layers.Dense(num_classes, activation='softmax'))

    return model




In [27]:
model_Alex=AlexNet((240,240,3),5)
model_Alex.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)
model_Alex.fit(trainDataset,epochs=10,batch_size=64)

Epoch 1/10
113/127 ━━━━━━━━━━━━━━━━━━━━ 12s 910ms/step - accuracy: 0.1973 - loss: 0.6935

KeyboardInterrupt: 